<a href="https://colab.research.google.com/github/abh156ek/AIAproject-FittingRoom/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install libraries**

In [1]:
!pip install diffusers accelerate
!pip install sentence-transformers
!pip install gradio
!pip install pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 13.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinu

In [2]:
from PIL import Image
import requests
from io import BytesIO
import torch
from sentence_transformers import SentenceTransformer, util
from IPython.display import display
from diffusers import AutoPipelineForInpainting, AutoencoderKL
from diffusers.utils import load_image
import gradio as gr




device = 'cuda' if torch.cuda.is_available() else 'cpu'
from sentence_transformers import SentenceTransformer


model = SentenceTransformer('sentence-transformers/clip-ViT-B-32', device=device)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
/usr/local/lib/python3.10/dist-packages/diffusers/models/vq_model.py:20: FutureWarning: `VQEncoderOutput` is deprecated and will be removed in version 0.31. Importing `VQEncoderOutput` from `diffusers.models.vq_model` is deprec

modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

0_CLIPModel/preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

0_CLIPModel/config.json:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

0_CLIPModel/vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

0_CLIPModel/special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

0_CLIPModel/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [3]:


def load_image(image_url):
    response = requests.get(image_url)
    return Image.open(BytesIO(response.content))

def encode_images(image_urls):
    images = [load_image(url) for url in image_urls]
    return model.encode(images, batch_size=8, convert_to_tensor=True, device=device)

def open_and_display_bw_image(image_url):
    response = requests.get(image_url)
    response.raise_for_status()
    image = Image.open(BytesIO(response.content))
    return image

def find_most_similar_image(text, image_dict):
    image_urls = list(image_dict.keys())
    text_embedding = model.encode(text, convert_to_tensor=True, device=device)
    image_embeddings = encode_images(image_urls)

    similarities = util.pytorch_cos_sim(text_embedding, image_embeddings)[0]

    most_similar_idx = torch.argmax(similarities).item()
    most_similar_image_url = image_urls[most_similar_idx]
    most_similar_item_link = image_dict[most_similar_image_url]

    return most_similar_image_url, most_similar_item_link


In [4]:

image_dict = {
    'https://m.media-amazon.com/images/I/81ek8zyFDOL._SX679_.jpg':'https://www.amazon.in/IndoPrimo-Cotton-Casual-Regular-Windopande/dp/B0CQR4CNSJ/ref=pd_rhf_se_s_pd_crcd_d_sccl_2_2/261-7345148-5939921?pd_rd_w=Qjfkm&content-id=amzn1.sym.785b16db-ca40-46a3-ae75-2b38bb48d1aa&pf_rd_p=785b16db-ca40-46a3-ae75-2b38bb48d1aa&pf_rd_r=D1E2GHR5DE2TGA4XQ03Y&pd_rd_wg=YNDPY&pd_rd_r=af385f3c-eec3-4310-8969-40248a4ffb69&pd_rd_i=B0CQR1FZKC&psc=1&tag=coa_in-21',
    'https://m.media-amazon.com/images/I/41QKjOKDFcL.jpg':'https://www.amazon.in/Leriya-Fashion-Textured-Shirts-Stylish/dp/B0CRDG857P?tag=coa_in-21',
    'https://m.media-amazon.com/images/I/71MaGTXl7ML._SY879_.jpg':'https://www.amazon.in/Lymio-Casual-Stylish-Printed-Mistry/dp/B0BRKJ3QSR/ref=sr_1_52?dib=eyJ2IjoiMSJ9.vslCe-0M-Q6hWPXBqX5Or1lFqjixy1c9679i9vmfbtQQ39C4WaU1a7BX0LGt9bovSJrXDbi0vmgQ6nkWeEHajmxRRcpMhiB0Ja&tag=coa_in-21',
    'https://m.media-amazon.com/images/I/71KCmMRyJiL._SX679_.jpg':'https://www.amazon.in/IndoPrimo-Cotton-Casual-Regular-Sleeves/dp/B0C9DD1BQL/ref=sr_1_22_sspa?dib=eyJ2IjoiMSJ9.vslCe-0M-Q6hWPXBqX5Or1lFqjixy1c9679i9vmfbtQQ39C4WaU1a7BX0LGt9bovSJrXDbi0vmgQ6nkWeEHajmxRRcpMhiB0Ja&tag=coa_in-21',
    'https://m.media-amazon.com/images/I/51TZSGQ63CL._SX679_.jpg':'https://www.amazon.in/ADRO-Mens-Printed-T-Shirt-Navy/dp/B09PNRBM72/ref=sr_1_23_sspa?crid=VMZZ64S58TXR&dib=eyJ2IjoiMSJ9.obhGuUO9S0v0lWQLZKP4O0_TXu79coTioe644Cq6w52BzoxcPl-xw4Mhw7nkQ-PoZDFlnL6st-OOAjYRMW3Ov-uldlYLmHAZ1KAyj0EziBf_FTsLkHnlLGeA7gyBpCgX-QNCYk10Kxy915sQeaYe-qJ3exaylNQffwEg2CLI8Cb-vi7VogH-vBYPM6pm2hlRgNyg_1T2kwkGb1oASCjcTB_0SZLtSKO8BDV9EOyQU43TgyKR63MrYW9VPjlRiWCL12niDMhnpqlYXxzh73s9rp7ACbV3oumQvtIimupXhpE.xBQs3t16msYz9Rtt2ZXM5MpFwFqMbU6poEzdJ5oC3zg&dib_tag=se&keywords=sports+wear&qid=1718893466&s=apparel&sprefix=sports+wea%2Capparel%2C229&sr=1-23-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9tdGY&psc=1',
    'https://m.media-amazon.com/images/I/61HfipQ+zrL._SY879_.jpg':'https://www.amazon.in/Leriya-Fashion-Shirts-Sleeves-Vacation/dp/B0C2JY5989?tag=coa_in-21',
    'https://m.media-amazon.com/images/I/61HfipQ+zrL._SY741_.jpg':'https://www.amazon.in/Lymio-Casual-Stylish-S-Kent-Maroon/dp/B0CTQNWNYZ/ref=sr_1_56?crid=30U1XYOHVIU&dib=eyJ2IjoiMSJ9.4VIR3ukzdTrzcj3PLIi7ULpR5CpQLq0PStidCOef8XsNfhxnd1pYUt8JerPv-4HwjEfABWYYY4_jMwxKQdDtr0sPjf2vhDKEzhgxzPgcWd6BGUBRyTC-gZxL8JEnY4qlDEtQIVV_PZyJch0n-Nu5xoeOr4r1yrXguaPRQnrtEsCOTVIBFhI8rtBsq42IMIVrT2jEQ78CZnZNF0HuMigzN-P3aIQGTF6HZOZ628F6rd3pe5DPFuPJfzMvpYUPipS_BXbVmxw65nqDBGt23GY_ke2S7TVht0kNgD_pnFwYy9Y.o4gdjQ9aJwNSIe8wM4BLdlgb63QpFlptdqEBIadefSE&dib_tag=se&keywords=shirt&qid=1716128577&sprefix=shir%2Caps%2C217&sr=8-56&tag=coa_in-21'


}
#text = "blue shirt"

#most_similar_image_url, most_similar_item_link = find_most_similar_image(text, image_dict)

#img_object = open_and_display_bw_image(most_similar_image_url)


**Import libraries**

**Load the pipeline**

In [5]:
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipeline = AutoPipelineForInpainting.from_pretrained("diffusers/stable-diffusion-xl-1.0-inpainting-0.1", vae=vae, torch_dtype=torch.float16, variant="fp16", use_safetensors=True).to("cuda")
pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin", low_cpu_mem_usage=True)

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'decay': 0.9999, 'inv_gamma': 1.0, 'min_decay': 0.0, 'optimization_step': 37000, 'power': 0.6666666666666666, 'update_after_step': 0, 'use_ema_warmup': False} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


ip-adapter_sdxl.bin:   0%|          | 0.00/703M [00:00<?, ?B/s]

sdxl_models/image_encoder/config.json:   0%|          | 0.00/2.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.69G [00:00<?, ?B/s]

**Load the main image**

**Load the clothing image**

**Create the mask image**

To create the mask image we're going to use a [body segmentation tool](https://github.com/TonyAssi/Segment-Body) that will do it automatically

In [9]:
! git clone https://github.com/TonyAssi/Segment-Body.git
%cd /content/Segment-Body
! pip install -r requirements.txt
! cp ./SegBody.py ..
%cd ..

Cloning into 'Segment-Body'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 40 (delta 10), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (40/40), 14.29 KiB | 4.76 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/Segment-Body
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.3 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp310-cp310-linux_x86_64.whl size=1055336 sha256=58

In [10]:
from SegBody import segment_body


download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:05<00:00, 55063.16KB/s]
/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


config.json:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/image_processing_segformer.py:103: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


**Generate image**

In [ ]:
"""pipeline.set_ip_adapter_scale(1.0)
images = pipeline(
    prompt="photorealistic, perfect body, beautiful skin, realistic skin, natural skin",
    negative_prompt="ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings",
    image=image,
    mask_image=mask_image,
    ip_adapter_image=ip_image,
    strength=0.99,
    guidance_scale=7.5,
    num_inference_steps=100,
).images
images[0]"""

In [ ]:
#print("The link to buy the item is:", most_similar_item_link)


In [11]:
def virtual_try_on(img, clothing_text):
    clothing_url, most_similar_item_link = find_most_similar_image(clothing_text, image_dict)
    clothing = load_image(clothing_url)

    _, mask_img = segment_body(img, face=False)

    pipeline.set_ip_adapter_scale(1.0)



    images = pipeline(
        prompt="photorealistic, perfect body, beautiful skin, realistic skin, natural skin",
        negative_prompt="ugly, bad quality, bad anatomy, deformed body, deformed hands, deformed feet, deformed face, deformed clothing, deformed skin, bad skin, leggings, tights, stockings",
        image=img,
        mask_image=mask_img,
        ip_adapter_image=clothing,
        strength=0.99,
        guidance_scale=7.5,
        num_inference_steps=100,
    ).images

    return images[0], most_similar_item_link

iface = gr.Interface(
    fn=virtual_try_on,
    inputs=[
        gr.Image(type="pil", label="Input Image"),
        gr.Textbox(label="Clothing Description or URL")
    ],
    outputs=[
        gr.Image(type="pil", label="Output Image"),
        gr.Textbox(label="Item Link")
    ]
)

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9018b16950f196230f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
